# Music Recommendor 


In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pickle 
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler

from matplotlib.lines import Line2D

In [11]:
kmeans = pickle.load(open('model.p', 'rb'))
transformer = pickle.load(open('transformer.p','rb'))

In [15]:
top100 = pd.read_csv('top100.csv')
Song_cluster = pd.read_csv('Song_cluster.csv')
Audio_features = pd.read_csv('Audio_features.csv')

In [19]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
string
string.split('\n')
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

secrets_dict

{'cid': '0c5b3b62ebe84fe5982f4181fcd53d2a',
 'csecret': '5b700b767ae04a02ab242ccf58fe93c7'}

In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [33]:
def search_track(sp, song_title):
    track = sp.search(q=song_title)
    track_uri = [track['tracks']['items'][0]['uri']]
    return track_uri

def get_audio_features(sp, track_uri):
    audio_features = sp.audio_features(track_uri)
    return audio_features

def create_audio_features_df(audio_features, transformer):
    audio_features_df = pd.DataFrame(audio_features)
    audio_feats = audio_features_df.select_dtypes(include='number')
    transformed_audio_feats = pd.DataFrame(transformer.transform(audio_feats), columns=audio_feats.columns)
    return transformed_audio_feats

def predict_track_cluster(kmeans, audio_feats):
    track_cluster = kmeans.predict(audio_feats)
    return track_cluster

def display_recommendations(df, track_cluster):
    rec = df[(df['cluster'] == track_cluster[0])].sample().reset_index(drop=True)
        #display(rec[['Track','Artist']])
        
    print('My recommendation is '+ str(rec['Track'][0]) + 'from the artist ' + str(rec['Artist'][0]))
     


In [38]:
song_title = input("Enter the song title: ")


Enter the song title: Flowers


In [39]:
x = random.randint(0,100)
# check if the song is in the DataFrame
if song_title in top100['title'].values:
    # if the song is in the DataFrame, recommend a different song
    recommended_song = top100.loc[top100['title'] != song_title, 'title'].iloc[x]
    print("This is a Hit now! You might like:", recommended_song)
    
else:
    track_uri = search_track(sp, "song_title")
    audio_features = get_audio_features(sp, track_uri)
    transformed_audio_feats = create_audio_features_df(audio_features, transformer)
    track_cluster = predict_track_cluster(kmeans, transformed_audio_feats)
    display_recommendations(Song_cluster, track_cluster)


This is a Hit now! You might like: One Thing At A Time
